### Thinking1
什么是近似最近邻查找，常用的方法有哪些

答：最近邻查找指根据数据的相似度，在数据库中查找与之最相近的数据，而最近邻查找的复杂度是线性的，因而不能满足大规模数据检索时对时间性能的要求，而近似最近邻查找指的是牺牲可接受范围的精确率以提高检索效率的近邻查找方式，常用的方法有：  

（1）基于哈希的方法   
以LSH（局部敏感哈希）为例：选取哈希函数将高维数据映射到低维空间，使得高维空间中相似的数据映射到同一个分桶的概率很大，反之则很小，检索时只需要对被映射到同一个分桶中的候选项进行检索。  

（2）基于量化的方法  
以乘积量化为例：将特征向量拆分成若干个子向量，分别对子向量进行聚类，然后将每个子向量所属的聚类中心的ID拼接以表达原来的特征向量。

### Thinking2
为什么两个集合的minhash值相同的概率等于这两个集合的Jaccard相似度

答：对于集合i，j，根据元素的有无情况可以得到两列特征Ci，Cj（两集合皆无的元素不予考虑），由于minhash值为将特征矩阵的行随机排列后，每列第一次出现1的行号，所以P(h(i)=h(j)) = P(Ci,Cj第一行同时为1) = P(Ci∩Cj元素个数/Ci∪Cj元素个数) = Jaccard相似度。

### Thinking3
SimHash在计算文档相似度的作用是怎样的？

答：SimHash在计算文档相似度的作用过程如下：  
&emsp;&emsp;（1）对每个文档提取特征向量以及对应的权重（如词与词频）；  
&emsp;&emsp;（2）通过hash函数计算各特征向量的hash值，其中hash值为指定位数的2进制数;  
&emsp;&emsp;（3）将各个特征进行对位加权，加权规则为：若该位的hash值为1，则加上对应权重，若该位的hash值为0，则减去对应权重；  
&emsp;&emsp;（4）所得的加权结果为正的位为1，反之为0，得到SimHash值作为文档的指纹；  
&emsp;&emsp;（5）计算两篇文档指纹的Hamming距离，距离在3以内则认为两篇文档相似度较高。

### Thinking4
为什么YouTube采用期望观看时间作为评估指标

答：首先单以点击率作为评估指标会具有一定的欺骗性，因为很多时候用户点开一个视频不代表他会喜欢一个视频，所以采用期望观看时长可以更精准地推荐，其次YouTube作为一个视频网站，不仅仅是以点击率为目标，而是希望用户能观看更长的时间，采用期望观看时间作为评估指标更符合商业应用场景。

### Action1
使用MinHashLSHForest对微博新闻句子进行检索 weibo.txt
针对某句话进行Query，查找Top-3相似的句子

In [152]:
# 载入数据
with open('weibos.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# 数据探索    
text[:200]

'#斯科拉里愿意执教国足#上一届如果是里皮从头芾到尾，是很大机会入世界杯的，这一届，没几个能用的，除非大力归化，谁来都没用。 \u200b\n国足输给叙利亚之后，里皮辞职。谁将成为新主帅，成为广大球迷关注的焦点。目前舆论方面，倾向于三个人：山东鲁能主帅李霄鹏、武汉卓尔主帅李铁、前广州恒大主帅斯科拉里。 \u200b\n据了解，无论中国足协态度如何，里皮其实在宣布请辞同时已经去意已决。据了解。比赛当晚，他的太太西蒙内塔女士及'

In [153]:
# 切分句子
import re
sentences = re.split('[。！？#]', re.sub('[\u200b\n“”]', '', text))
sentences = [i for i in sentences if len(i)!=0]
sentences[:5]

['斯科拉里愿意执教国足',
 '上一届如果是里皮从头芾到尾，是很大机会入世界杯的，这一届，没几个能用的，除非大力归化，谁来都没用',
 ' 国足输给叙利亚之后，里皮辞职',
 '谁将成为新主帅，成为广大球迷关注的焦点',
 '目前舆论方面，倾向于三个人：山东鲁能主帅李霄鹏、武汉卓尔主帅李铁、前广州恒大主帅斯科拉里']

In [154]:
# 加载停用词
with open('chinese_stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = [i.strip() for i in f.readlines()]
    
# 分词
import jieba
def split_text(sentence):
    words = jieba.cut(sentence)
    result = ' '.join([w for w in words if w not in stopwords])
#     result = ' '.join(list(words))
    return result

from collections import defaultdict
doc = defaultdict()
for i in sentences:
    split = split_text(i)
    doc[split] = i

In [157]:
# 创建minhash对象以及LSH Forest
from datasketch import MinHash, MinHashLSHForest
def get_minhash(sentence):
    temp = MinHash()
    for i in sentence:
        temp.update(i.encode('utf-8'))
    return temp

minhash_list = defaultdict()
forest = MinHashLSHForest()
for i in doc:
    temp = get_minhash(i)
    minhash_list[i] = temp
    forest.add(i, temp)

# 建立索引，方便检索
forest.index()

In [159]:
query = '斯科拉里执教国足'
query_split = split_text(query)
query_minhash = get_minhash(query_split)

# 查询Top-3相似句子
result = forest.query(query_minhash,3)
# print(result)
for i in range(len(result)):
    print(doc[result[i]].replace(' ',''), '\n相似度为:', query_minhash.jaccard(minhash_list[result[i]]))

大家支持斯科拉里不 
相似度为: 0.3046875
因为中国队给斯科拉里开足了薪水，给足了他需要的条件 
相似度为: 0.4375
斯科拉里愿意执教国足 
相似度为: 0.8671875
